In [ ]:
###############
### install ###
###############
'''
pip install pandas
pip install xlrd
'''

In [ ]:
##############################################################
### cleaning of city data, deletion of duplicate city data ###
##############################################################

import pandas as pd
dfcity = pd.read_csv('worldcities.csv')              #Reading the csv file and storing it in a dataframe called dfcity

i = 0                                                    #i and e are the indexes of the cities in the dataframe dfcity
e = 0
to_drop = []                                             #creation of an empty list

for city in dfcity['city']:
    for city2 in dfcity['city']:
        if city == city2 :                               #Comparing the city names in the dataframe dfcity
            if i != e :                                  #if the index of the 2 citie with the same name is different
                pope = dfcity.population[e]
                popi = dfcity.population[i]
                
                if pope > popi:   
                    to_drop.append(i)                    #for these 2 cities with the same name we add the index of the city with the least population to the list 
                    
                elif popi>pope:
                    to_drop.append(e)                     #for these 2 cities with the same name we add the index of the city with the least population to the list 
                    
        e += 1                                             #add 1 to e to keep the index updated
    i += 1                                                 #add 1 to i to keep the index updated
    e = 0                                                  #reset the index to 0 as the loop starts again



to_drop = list(set(to_drop))                               #many cities in the world have the same name so we delete duplicate elements
to_drop.sort()                                             #It sorts the list in ascending order


for i in (to_drop):
    dfcity=dfcity.drop(i)                                  #Dropping the cities with the same name and the least population.

dfcity.to_csv('data_city_update.csv')                      #Saving the dataframe dfcity in a csv file called data_city_update.csv

In [ ]:
###########################################
### delete tweet lines without location ###
###########################################

low_memory=False
df = pd.read_csv('data_twitter_api.csv',engine='python', error_bad_lines=False)   #reading of the two dataset, data with tweet and data of the city
dfcity = pd.read_csv('data_city_update.csv')

try:                                                                               #Trying to remove the column 'Unnamed: 0' from the dataframe dfcity. If it fails, it will pass. Because pd.read_csv creates a column 'Unnamed: 0'
    dfcity.pop('Unnamed: 0')                                                       #Because pd.read_csv creates a column 'Unnamed: 0'
except KeyError as err:
        pass

to_fast=len(df)                                                                    #if the data contains too many tweets we can access it by reducing the data by modifying len(df) by the number of data we want
df_fast = df[:to_fast]

i=0                                                                                 #The index of the first element 

while i<len(df_fast):
    try :
        if ',' in df_fast.user_location[i]:                             
            text =df_fast.user_location[i]
            head, sep, tail = text.partition(',')
            df_fast.user_location[i]=head                                          #Removing the comma and after from the user_location column

    except TypeError as err:                                                       #A way to handle the error. If the program encounters a TypeError, it will pass and continue the program
        pass

    i+=1                                                                           #add 1 to e to keep the index updated


e=0                                                                                #The index of the first element 
twi=[]                                                                             #creation of an empty list

for i in df_fast['user_location']:                                                 #loop that will look at each location of user_location in the tweet data set
    
    for a in dfcity['city']:                                                       #another loop that will look at each location of city in the dataset of the world city
        if i == a:                                                                 #check whether the two elements are the same
            twi.append(e)                                                          #if yes, we add e (index element) of the line in data set of tweet
    e+=1                                                                           #add 1 to e to keep the index updated
    
twi = list(set(twi))                                                               #many cities in the world have the same name so we delete duplicate elements
twi.sort()                                                                         #It sorts the list of index of the tweet with the correct location

nb_tweet_in_data = len(twi)-1                                                      #Taking the last index of the list of the index of the tweet with the correct location
df_s = df_fast[:twi[nb_tweet_in_data]]

to_drop = [ele for ele in range(len(df_s)) if ele not in twi]                      #we take the inverse of the elements to know which are the elements without corect localization in order to remove them from the data
to_drop.sort()

for i in (to_drop):                                                                #loop that will drop the line without location in the dataset
    df_s=df_s.drop(i)
    

'''
with open("listTwi.txt", "w") as f:       #Writing the list of the index of the tweet with the correct location in a txt file
    for s in twi:
        f.write(str(s) +",")
'''

df_s.to_csv('data_5K.csv')                                                          #Saving the dataframe df_s to a csv file called data_5K.csv


In [ ]:
##########################################################################
### Adding the latitude and longitude for each location for each tweet ###
##########################################################################

import pandas as pd

df = pd.read_csv('data_5K.csv')                #Reading the csv file
dfcity = pd.read_csv('data_city_update.csv')   #Reading the csv file

try:                                           #Trying to remove the column 'Unnamed: 0' from the dataframe dfcity. If it fails, it will pass. Because pd.read_csv creates a column 'Unnamed: 0'
    df.pop('Unnamed: 0')
except KeyError as err:
        pass

e=0                                            ##The index of the first element
latitude=[]                                    #creation of an empty list which will be used to indicate the latitude assigned to the city of a tweet
longitude=[]                                   #creation of an empty list which will be used to indicate the longitude assigned to the city of a tweet


for location in df['user_location']:           #This is a loop that is going through the user_location column of the dataframe df
    while location != dfcity.city[e]:          #comparing it to the city column of the dataframe dfcity
        e+=1                                   #add 1 to e to keep the index updated
    latitude.append(dfcity.lat[e])             #if the while loop stops it means that the right city is found, the latitude and longitude of the city is appended to the latitude and longitude list
    longitude.append(dfcity.lng[e])
    


    e=0                                        #reset the index to 0 as the loop starts again
    

df.insert(6,"U_latitude",latitude,True)        #Inserting the latitude and longitude list into the dataframe df at the 6th and 7th column
df.insert(7,"U_longitude",longitude,True)


df.to_csv('data_tweet_with_location.csv')      #Saving the dataframe df into a csv file called tweet_with_location.csv

In [ ]:
#########################################################################
### Converting the date and time from excel format to datetime format ###
#########################################################################

import pandas as pd
import xlrd

df = pd.read_csv('data_tweet_with_location.csv') 

try:                                                            #Trying to remove the column 'Unnamed: 0' from the dataframe dfcity. If it fails, it will pass. Because pd.read_csv creates a column 'Unnamed: 0'
    df.pop('Unnamed: 0')
except KeyError as err:
        pass

a=0                                                             #The index of the first element
while a < len(df):                                              #The above code is converting the date and time from excel format to datetime format
    try:
        user_created_date = xlrd.xldate_as_datetime(df.user_created[a], 0)
        df.user_created[a]=user_created_date
        tweet_date = xlrd.xldate_as_datetime(df.date[a], 0)
        df.date[a]=tweet_date
    except TypeError as err:
        pass
    
    a+=1                                                         #add 1 to e to keep the index updated


df.to_csv('data_tweet_with_location_and_date.csv')               #Saving the dataframe df to a csv file

In [ ]:
##################################################
######## extract the hour from the date ##########
### extract the tweet link from the tweet text ###
##################################################

df = pd.read_csv('data_tweet_with_location_and_date.csv') 

try:                                                    #Trying to remove the column 'Unnamed: 0' from the dataframe dfcity. If it fails, it will pass. Because pd.read_csv creates a column 'Unnamed: 0'
    df.pop('Unnamed: 0')
except KeyError as err:
        pass

hour = []                                               #Creating empty lists to store data 
date = []
tweet = []
link = []

i=0                                                      #The index of the first element
while i < len(df):
    date_hour =df.date[i]
    head, sep, tail = date_hour.partition(' ')          #Splitting the string date_hour into three parts: head wich corespond to the date, sep and tail which corresponds to the hour. The separator is the space ' '
    date.append(head)                                   #appends the date to the date list
    head, sep, tail=tail.partition('+')                 #It splits the string tail into three parts: head wich corespond to the hour, sep and tail which corresponds to the time zone wich is which is incorrect information. The separator is the '+'
    hour.append(head)                                   #It appends the hour to the hour list                         

    tw =df.text[i]                                      #Creating a variable tw which is equal to the text of the tweet at the index i
    
    head, sep, tail = tw.partition('https://t.co/')     #It splits the string tw into three parts: head wich corespond to the text of the tweet, sep and tail which corresponds to the link. The separator is the 'https://t.co/'
    
    tweet.append(head)                                  #It appends the text of the tweet to the tweet list
    
    if 'https://t.co/' in tail :                        #Checking if there is stil a link in the tail of the tweet text. If there is this means that there is a link in the tweet before the twitter link of the tweet itself
        lk = tail
        head, sep, tail = lk.partition('https://t.co/')

    link.append(sep+tail)                               #It appends the link to the link list
    
    i+=1                                                #add 1 to e to keep the index updated



df.pop('date')                                          #Removing the column old 'date'
df.insert(10,"date",date,True)                          #inserting the new columns 'date' and 'hour' from the list at the index 10 and 11
df.insert(11,"hour",hour,True)
df.pop('text')                                          #Removing the column old 'text'
df.insert(12,"tweet",tweet,True)                        #Inserting the new columns 'tweet' and 'link' from the list at the index 12 and 13
df.insert(13,"link",link,True)

df.to_csv('data_tweet_with_location_and_date_hour.csv')  #Saving the dataframe df into a csv file called 'data_tweet_with_location_and_date_hour.csv'